In [ ]:
//:dep plotly = { version = ">=0.7.0" }

In [ ]:
:dep plotly = { path = "<localpath>/plotly/plotly", features = ["kaleido", "plotly_ndarray", "plotly_image"]}
:dep itertools-num = "0.1.3"
:dep reqwest = { version = "0.11", features = ["blocking"] }
:dep image = "0.24.2"

In [ ]:
use plotly::{Layout, Plot, Trace};

fn simple_plot<T: Trace + 'static>(trace: Box<T>, modify: fn(Layout) -> Layout) {
    let mut plot = Plot::new();
    plot.add_trace(trace);
    let mut layout = Layout::new().height(800);
    plot.set_layout(modify(layout));
    plot.lab_display();
}

## Scatter plot example

In [ ]:
use itertools_num::linspace;
use plotly::{common::Mode, Scatter};

fn demo_scatter_trace() -> Box<Scatter<f64, f64>> {
    let n: usize = 100;
    let t: Vec<f64> = linspace(0., 10., n).collect();
    let y: Vec<f64> = t.iter().map(|x| x.sin()).collect();
    Scatter::new(t, y).mode(Mode::Markers)
}

simple_plot(demo_scatter_trace(), |layout| layout);

## 3D model mesh example

In [ ]:
#[derive(Clone, Debug, Default)]
struct MeshData {
    x: Vec<f64>,
    y: Vec<f64>,
    z: Vec<f64>,
    
    i: Vec<usize>,
    j: Vec<usize>,
    k: Vec<usize>,
}

fn download_model(url: &str) -> MeshData {
    let mut mesh = MeshData::default();
    for line in reqwest::blocking::get(url).unwrap().text().unwrap().split("\n") {
        let words = line.split(" ").collect::<Vec<&str>>();
        match words[0] {
            "v" => {
                mesh.x.push(words[1].parse().unwrap());
                mesh.y.push(words[2].parse().unwrap());
                mesh.z.push(words[3].parse().unwrap());
            },
            "vn" => (),
            "f" => {
                mesh.i.push(words[1].parse::<usize>().unwrap() - 1);
                mesh.j.push(words[2].parse::<usize>().unwrap() - 1);
                mesh.k.push(words[3].parse::<usize>().unwrap() - 1);
            },
            _ => ()
        }
    }
    mesh
}

let bunny = download_model("https://raw.githubusercontent.com/alecjacobson/skeleton-builder/master/big-buck-bunny.obj");

In [ ]:
use plotly::Mesh3D;

fn demo_mesh_trace(mesh: &MeshData) -> Box<Mesh3D<f64,f64, f64>> {
    let MeshData { x, y, z, i, j, k } = mesh.clone();
    let intensity = x.clone();
    Mesh3D::new(x, z, y, i, j, k).intensity(intensity)
}

simple_plot(demo_mesh_trace(&bunny), |layout| layout);

## Image example

In [ ]:
use image::RgbImage;

fn download_image(url: &str) -> RgbImage {
    let img_bytes = reqwest::blocking::get(url).unwrap().bytes().unwrap();
    image::load_from_memory(&img_bytes).unwrap().into_rgb8()
}

let tux = download_image("https://upload.wikimedia.org/wikipedia/commons/thumb/3/35/Tux.svg/405px-Tux.svg.png");

In [ ]:
use plotly::{Image, image::{PixelColor, ZSmooth}};

fn demo_image_trace(img: &RgbImage) -> Box<Image<u8>> {
    let width = img.width() as usize;
    let height = img.height() as usize;
    let mut arr: Vec<Vec<PixelColor<u8>>> =
        vec![vec![PixelColor::Color3([0, 0, 0]); width]; height];
    for x in 0..width {
        for y in 0..height {
            let mut rgb = [0, 0, 0];
            for k in 0..3 {
                 rgb[k] = img[(x as u32, y as u32)][k];
            }
            arr[y][x] = PixelColor::Color3(rgb);
        }
    }
    Image::new(arr)
}

simple_plot(demo_image_trace(&tux), |layout| layout);

## Mapbox example

In [ ]:
use plotly::{ScatterMapbox, layout::{Center, HoverMode, Mapbox}, common::Marker};

fn demo_scattermapbox_trace() -> Box<ScatterMapbox<f64, f64>> {
    let lat = vec![45.5017];
    let lon = vec![-73.5673];
    let size = 14;
    let text = "Montreal";
    
    ScatterMapbox::new(lat, lon)
        .mode(Mode::Markers)
        .marker(Marker::new().size(size))
        .text(text)
}

simple_plot(demo_scattermapbox_trace(), |layout: Layout| {
    layout
        .hover_mode(HoverMode::Closest)
        .mapbox(Mapbox::new()
        .access_token("pk.eyJ1Ijoiam9lbHNqb2dyZW4iLCJhIjoiY2tnNmFnd3JnMDA0bTJybjZidzUxaWIyNCJ9.p_L0C7GQWe4gog_wQf6OrA")
        .bearing(0.0)
        .center(Center::new(45.0, -73.0))
        .pitch(0.0)
        .zoom(5.0))
});